In [181]:
## import the liberaries
import pandas as pd
import numpy as np
import seaborn as sns

In [182]:
## Read all the sheets
df_sheet1= pd.read_csv(r'C:\Users\arvinmvn\Downloads\DA_Timeline_Metrics_1627969497093.csv',)
df_sheet2= pd.read_excel(r'C:\Users\arvinmvn\Downloads\npt.xlsx',)
df_sheet3= pd.read_excel(r'C:\Users\arvinmvn\Downloads\Week 25 tmda.xlsx',sheet_name='w29')

In [183]:
## clean all sheets

##clean DA timeline sheet 1

## Steps done
# sort DAs in low to high
# remove process counts 0
## delete column D

## code for above steps ##

df_sheet1=df_sheet1.sort_values(by=['DA Login']) # sort sheet 1 by DA login
df_sheet1= df_sheet1.dropna(subset=['First Login Time']) ## drop First login time NA 
df_sheet1=df_sheet1[df_sheet1['Processed Units']!=0] ## remove all the process counts 0
df_sheet1=df_sheet1.drop(['Skill Value ID'], axis=1) ## remove skill value id column

## cleaning NPT sheet 2 
df_sheet2= df_sheet2.drop(columns=['Idle No Data']) ## drop idle no data column as it is present in sheet 1

## Clean DA vs TM sheet 3

df_sheet3=df_sheet3.rename(columns={"DA Alias": "DA Login"}) # normalize name for sheet 3 DA login, same subset as DA Alias

In [184]:
## Merge sheet 1, 2 and 3

df_merge = pd.merge(df_sheet1, df_sheet2, on="DA Login",how="inner")

#df_merge = pd.merge(df_merge, df_sheet2, on="DA Login",how="inner")
df_merge=pd.merge(df_merge,df_sheet3,on="DA Login",how="inner")
df_merge=df_merge.drop_duplicates()
print(df_merge.columns)

Index(['DA Login', 'First Login Time', 'Current Status', 'UPH', 'AHT (In sec)',
       'Velocity', 'Utilization %', 'NPT %', 'Processed Units',
       'Production Hours', 'Non Mercury Production Hours', 'DA LoggedIn Hr',
       'Processed Hours', 'Manual Skip Hours', 'Non Mercury Processing Hours',
       'Staff Hours', 'NPT', 'Idle No Data', 'Training Hours', 'Lost Hours',
       'System Issue', 'System Latency', 'Schedule Non Adherence', 'Training',
       'Excess Break', 'Overturned Review', 'Meeting', 'Level 1 Manager'],
      dtype='object')


In [186]:
## Performance Metrics Table
table_PM= pd.pivot_table(df_merge, values=['DA LoggedIn Hr', 'Processed Hours','Processed Units','AHT (In sec)','Velocity','Manual Skip Hours','Lost Hours'],
                         index=['Level 1 Manager'], 
                         columns= [] ,margins=False,
                         margins_name='total', aggfunc=np.mean , fill_value=0)
#pd.set_option('display.max_rows', df_merge.shape[0]+1)
table_PM.round(2)

,AHT (In sec),DA LoggedIn Hr,Lost Hours,Manual Skip Hours,Processed Hours,Processed Units,Velocity
Level 1 Manager,,,,,,,
czvij,60.00,3.64,4.24,0.08,3.19,237.62,74.81
danijos,48.86,4.00,4.01,0.07,3.59,288.75,84.14


In [187]:
## NPT Report
table_NPT= pd.pivot_table(df_merge, values=['Excess Break', 'Meeting','Training','Overturned Review'],
                         index=['Level 1 Manager'], 
                         columns= [] ,margins=False,
                         margins_name='total',aggfunc= np.sum , fill_value=0)
#pd.set_option('display.max_rows', df_sheet2.shape[0]+1)
table_NPT["sum"] = table_NPT.sum(axis=1)
table_NPT.round(2)

,Excess Break,Meeting,Overturned Review,Training,sum
Level 1 Manager,,,,,
czvij,0,0.00,3.05,6.59,9.64
danijos,0,1.04,1.27,2.62,4.93


In [188]:
## Lost Hours Report
df_merge["Idle"]=0
table_LH= pd.pivot_table(df_merge, values=['Idle','Idle No Data','System Latency'],
                         index=['Level 1 Manager'], 
                         columns= [] ,margins=False,
                         margins_name='total',aggfunc= np.sum , fill_value=0)
#pd.set_option('display.max_rows', df_sheet2.shape[0]+1)
table_LH["Total"] = table_LH.sum(axis=1)
table_LH.round(2)

,Idle,Idle No Data,System Latency,Total
Level 1 Manager,,,,
czvij,0,0.02,0.19,0.20
danijos,0,0.00,0.33,0.33


In [189]:
#NPT Utilization Report

table_UR= pd.pivot_table(df_merge, values=['Utilization %','NPT %'],
                         index=['Level 1 Manager'], 
                         columns= [] ,margins=False,
                         margins_name='total',aggfunc= np.sum , fill_value=0)
#pd.set_option('display.max_rows', df_merge.shape[0]+1)
table_UR["Total"] = table_UR.sum(axis=1)
table_UR.round(2)

,NPT %,Utilization %,Total
Level 1 Manager,,,
czvij,0.39,6.72,7.11
danijos,0.36,9.45,9.81


In [192]:
## Bulk Mail

import win32com.client as win32
outlook = win32.Dispatch('outlook.application')

    
for j in range (0,len(table_PM.index.get_level_values('Level 1 Manager'))):
    
    table_PM= pd.pivot_table(df_merge, values=['DA LoggedIn Hr', 'Processed Hours','Processed Units','AHT (In sec)','Velocity','Manual Skip Hours','Lost Hours'],
                         index=['Level 1 Manager'], 
                         columns= [] ,margins=False,
                         margins_name='total', aggfunc=np.mean , fill_value=0)
    
    table_NPT= pd.pivot_table(df_merge, values=['Excess Break', 'Meeting','Training','Overturned Review'],
                         index=['Level 1 Manager'], 
                         columns= [] ,margins=False,
                         margins_name='total',aggfunc= np.mean , fill_value=0)
    table_NPT["sum"] = table_NPT.sum(axis=1)
    
    table_LH= pd.pivot_table(df_merge, values=['Idle','Idle No Data','System Latency'],
                         index=['Level 1 Manager'], 
                         columns= [] ,margins=False,
                         margins_name='total',aggfunc= np.mean, fill_value=0)

    table_LH["Total"] = table_LH.sum(axis=1)
    
    table_UR= pd.pivot_table(df_merge, values=['Utilization %','NPT %'],
                         index=['Level 1 Manager'], 
                         columns= [] ,margins=False,
                         margins_name='total',aggfunc= np.mean , fill_value=0)

    pd.set_option('display.max_rows', df_merge.shape[0]+1)

    mail = outlook.CreateItem(0)

    mail.Subject = 'Test Report'
    mail.To='arvinmvn@amazon.com'
    #mail.To = table_PM.index.get_level_values('DA Login')[j]+'@amazon.com' 
    #mail.CC = table_PM.index.get_level_values('Level 1 Manager')[j] + '@amazon.com'
        
    
## i-> position of data in dataframe
    mail.HTMLBody = f"<h2>{'Hi ' + table_PM.index.get_level_values('Level 1 Manager')[j] + ','}<h2>"   + '''<h3> Please find your reports below <h3>'''  + '''<h3>Performance Metrics Report.</h3>
                   {}'''.format(table_PM[j:j+1].style.set_table_styles(
[{'selector': 'th',
  'props': [('background', '#F5F5F5'), 
            ('color', 'black'),
            ('font-family', 'verdana'),
            ('border-style','double'),
            ('border-width','thin'),
           ('font-weight', 'normal'),
           ('text-align','center')]},
 
 {'selector': 'td',
  'props': [('font-family', 'verdana'),
           ('border-style','double'),
            ('border-width','thin'),
           ('text-align','center'),
           ('text-align','center')]},

 {'selector': 'tr:nth-of-type(odd)',
  'props': [('background', 'white')]}, 
 
 {'selector': 'tr:nth-of-type(even)',
  'props': [('background', 'white')]},
 
 {'selector': 'tr:hover',
  'props': [('background-color', 'yellow')]}

]
).set_precision(2).render()) + '''<h3>Lost Hours Report.</h3>
                   {}'''.format(table_LH[j:j+1].style.set_table_styles(
[{'selector': 'th',
  'props': [('background', '#F5F5F5'), 
            ('color', 'black'),
            ('font-family', 'verdana'),
            ('border-style','double'),
            ('border-width','thin'),
           ('font-weight', 'normal'),
           ('text-align','center')]},
 
 {'selector': 'td',
  'props': [('font-family', 'verdana'),
           ('border-style','double'),
            ('border-width','thin'),
           ('text-align','center'),
           ('text-align','center')]},

 {'selector': 'tr:nth-of-type(odd)',
  'props': [('background', 'white')]}, 
 
 {'selector': 'tr:nth-of-type(even)',
  'props': [('background', 'white')]},
 
 {'selector': 'tr:hover',
  'props': [('background-color', 'yellow')]}

]
).set_precision(2).render()) + '''<h3>NPT Report.</h3>
                   {}'''.format(table_NPT[j:j+1].style.set_table_styles(
[{'selector': 'th',
  'props': [('background', '#F5F5F5'), 
            ('color', 'black'),
            ('font-family', 'verdana'),
            ('border-style','double'),
            ('border-width','thin'),
           ('font-weight', 'normal'),
           ('text-align','center')]},
 
 {'selector': 'td',
  'props': [('font-family', 'verdana'),
           ('border-style','double'),
            ('border-width','thin'),
           ('text-align','center'),
           ('text-align','center')]},

 {'selector': 'tr:nth-of-type(odd)',
  'props': [('background', 'white')]}, 
 
 {'selector': 'tr:nth-of-type(even)',
  'props': [('background', 'white')]},
 
 {'selector': 'tr:hover',
  'props': [('background-color', 'yellow')]}

]
).set_precision(2).render()) + '''<h3>Utilization Report.</h3>
                   {}'''.format(table_UR[j:j+1].style.set_table_styles(
[{'selector': 'th',
  'props': [('background', '#F5F5F5'), 
            ('color', 'black'),
            ('font-family', 'verdana'),
            ('border-style','double'),
            ('border-width','thin'),
           ('font-weight', 'normal'),
           ('text-align','center')]},
 
 {'selector': 'td',
  'props': [('font-family', 'verdana'),
           ('border-style','double'),
            ('border-width','thin'),
           ('text-align','center'),
           ('text-align','center')]},

 {'selector': 'tr:nth-of-type(odd)',
  'props': [('background', 'white')]}, 
 
 {'selector': 'tr:nth-of-type(even)',
  'props': [('background', 'white')]},
 
 {'selector': 'tr:hover',
  'props': [('background-color', 'yellow')]}

]
).set_precision(2).render())
    
    ## : i + 1 increment 1 of index 
# To attach a file to the email (optional):
#attachment  = "Path to the attachment"
#mail.Attachments.Add(attachment)

    mail.Send()

In [ ]:
## For Quality

In [258]:
## Read File
df_sheet4= pd.read_excel(r'C:\Users\arvinmvn\Desktop\Hourly Automation ADS\ADS vs ASR Quality Report Week 28 to 32 Aug 11.xlsx',sheet_name='RAW')
df_sheet4.head()

,SITE,LOCALE,CUST,SOID,TM,DA,ASR SCORE,DUR,DEV,Q TYPE,...,Processed Count,Quality Measured Count,Sample %,ASR vs Gold,ADS vs Gold,Difference,ADS vs Gold Overturn,ADS vs Gold Token,ASR vs Gold Overturn,ASR vs Gold Token
0,HYD,EN_US,ALEXA AUTOMOTIVE,SO-5002,ahja,estarlaa,800-900,1-3 s,Multiple Devices,NaN,...,1,0,0.0,NaN,NaN,NaN,0,NaN,0,NaN
1,HYD,EN_US,ALEXA AUTOMOTIVE,SO-5002,ahja,kaleabdu,800-900,1-3 s,Multiple Devices,NaN,...,1,0,0.0,NaN,NaN,NaN,0,NaN,0,NaN
2,HYD,EN_US,ALEXA AUTOMOTIVE,SO-5002,ahja,schikte,700-800,3-5 s,Multiple Devices,NaN,...,1,0,0.0,NaN,NaN,NaN,0,NaN,0,NaN
3,HYD,EN_US,ALEXA AUTOMOTIVE,SO-5002,ahja,spulig,800-900,1-3 s,Multiple Devices,NaN,...,1,0,0.0,NaN,NaN,NaN,0,NaN,0,NaN
4,HYD,EN_US,ALEXA AUTOMOTIVE,SO-5002,matlapud,alimela,800-900,1-3 s,Multiple Devices,NaN,...,1,0,0.0,NaN,NaN,NaN,0,NaN,0,NaN


In [259]:
df_sheet4.insert(1, "ADS Quality","")
df_sheet4.insert(2, "ASR Quality","")
df_sheet4=df_sheet4.rename(columns={"DA": "DA Login"})
df_sheet4=df_sheet4.rename(columns={"TM": "Level 1 Manager"})

In [386]:
table_ADS_Quality = pd.pivot_table(df_sheet4, values=['ADS vs Gold Overturn','ADS vs Gold Token'],index=['Level 1 Manager','ADS Quality','CUST'], 
                         columns= ['WEEK'] ,margins=True,
                         margins_name='total', aggfunc=np.sum , fill_value=0)

table_ASR_Quality= pd.pivot_table(df_sheet4, values=['ASR vs Gold Overturn','ASR vs Gold Token'],index=['Level 1 Manager','ASR Quality','CUST'], 
                         columns= ['WEEK'] ,margins=True,
                         margins_name='total', aggfunc=np.sum , fill_value=0)
table_ADS_Quality = (1- table_ADS_Quality['ADS vs Gold Overturn']/table_ADS_Quality['ADS vs Gold Token'])
table_ASR_Quality = (1- table_ASR_Quality['ASR vs Gold Overturn']/table_ASR_Quality['ASR vs Gold Token'])

#table_Quality_final =pd.merge(table_ADS_Quality,table_ASR_Quality, on = 'Level 1 Manager', how = 'inner')
#table_Quality_final= 100*table_Quality_final.round(4)
table_ASR_Quality = 100*table_ASR_Quality.round(4)
table_ADS_Quality = 100*table_ADS_Quality.round(4)
table_ASR_Quality
table_ADS_Quality
table_Variance =100*(table_ADS_Quality - table_ASR_Quality).round(4)

In [404]:
list = df_sheet4['Level 1 Manager'].unique().tolist()

In [ ]:
table_ADS_Quality

In [409]:
## Bulk Mail

import win32com.client as win32
outlook = win32.Dispatch('outlook.application')

    
for j in range (0,len(list)):
    
    table_ADS_Quality= pd.pivot_table(df_sheet4[df_sheet4['Level 1 Manager']==list[j]], values=['ADS vs Gold Overturn','ADS vs Gold Token'],index=['Level 1 Manager','ADS Quality','CUST'], 
                         columns= ['WEEK'] ,margins=True,
                         margins_name='total', aggfunc=np.sum , fill_value=0)

    table_ASR_Quality= pd.pivot_table(df_sheet4[df_sheet4['Level 1 Manager']==list[j]], values=['ASR vs Gold Overturn','ASR vs Gold Token'],index=['Level 1 Manager','ASR Quality','CUST'], 
                         columns= ['WEEK'] ,margins=True,
                         margins_name='total', aggfunc=np.sum , fill_value=0)
    table_ADS_Quality = (1- table_ADS_Quality['ADS vs Gold Overturn']/table_ADS_Quality['ADS vs Gold Token'])
    table_ASR_Quality = (1- table_ASR_Quality['ASR vs Gold Overturn']/table_ASR_Quality['ASR vs Gold Token'])

#table_Quality_final =pd.merge(table_ADS_Quality,table_ASR_Quality, on = 'Level 1 Manager', how = 'inner')
#table_Quality_final= 100*table_Quality_final.round(4)
    table_ASR_Quality = 100*table_ASR_Quality.round(4)
    table_ADS_Quality = 100*table_ADS_Quality.round(4)
    table_ASR_Quality
    table_ADS_Quality
    table_Variance =100*(table_ADS_Quality - table_ASR_Quality).round(4)
    table_Variance
    pd.set_option('display.max_rows', table_ADS_Quality.shape[0]+1)

    mail = outlook.CreateItem(0)

    mail.Subject = 'Test Report'
    mail.To='arvinmvn@amazon.com'
    #mail.To = table_PM.index.get_level_values('DA Login')[j]+'@amazon.com' 
    #mail.CC = table_PM.index.get_level_values('Level 1 Manager')[j] + '@amazon.com'
        
    
## i-> position of data in dataframe
    mail.HTMLBody = f"<h2>{'Hi ' + list[j] + ','}<h2>"   + '''<h3> Please find your Quality reports below <h3>'''  + '''<h3>Performance Metrics Report.</h3>
                   {}'''.format(table_ASR_Quality.style.set_table_styles(
[{'selector': 'th',
  'props': [('background', '#F5F5F5'), 
            ('color', 'black'),
            ('font-family', 'verdana'),
            ('border-style','double'),
            ('border-width','thin'),
           ('font-weight', 'normal'),
           ('text-align','center')]},
 
 {'selector': 'td',
  'props': [('font-family', 'verdana'),
           ('border-style','double'),
            ('border-width','thin'),
           ('text-align','center'),
           ('text-align','center')]},

 {'selector': 'tr:nth-of-type(odd)',
  'props': [('background', 'white')]}, 
 
 {'selector': 'tr:nth-of-type(even)',
  'props': [('background', 'white')]},
 
 {'selector': 'tr:hover',
  'props': [('background-color', 'yellow')]}

]
).set_precision(2).render()) + '''<h3>ADS Quality WoW.</h3>
                   {}'''.format(table_ADS_Quality.style.set_table_styles(
[{'selector': 'th',
  'props': [('background', '#F5F5F5'), 
            ('color', 'black'),
            ('font-family', 'verdana'),
            ('border-style','double'),
            ('border-width','thin'),
           ('font-weight', 'normal'),
           ('text-align','center')]},
 
 {'selector': 'td',
  'props': [('font-family', 'verdana'),
           ('border-style','double'),
            ('border-width','thin'),
           ('text-align','center'),
           ('text-align','center')]},

 {'selector': 'tr:nth-of-type(odd)',
  'props': [('background', 'white')]}, 
 
 {'selector': 'tr:nth-of-type(even)',
  'props': [('background', 'white')]},
 
 {'selector': 'tr:hover',
  'props': [('background-color', 'yellow')]}

]
).set_precision(2).render()) + '''<h3>Variance.</h3>
                   {}'''.format(table_Variance.style.set_table_styles(
[{'selector': 'th',
  'props': [('background', '#F5F5F5'), 
            ('color', 'black'),
            ('font-family', 'verdana'),
            ('border-style','double'),
            ('border-width','thin'),
           ('font-weight', 'normal'),
           ('text-align','center')]},
 
 {'selector': 'td',
  'props': [('font-family', 'verdana'),
           ('border-style','double'),
            ('border-width','thin'),
           ('text-align','center'),
           ('text-align','center')]},

 {'selector': 'tr:nth-of-type(odd)',
  'props': [('background', 'white')]}, 
 
 {'selector': 'tr:nth-of-type(even)',
  'props': [('background', 'white')]},
 
 {'selector': 'tr:hover',
  'props': [('background-color', 'yellow')]}

]
).set_precision(2).render())    
    ## : i + 1 increment 1 of index 
# To attach a file to the email (optional):
#attachment  = "Path to the attachment"
#mail.Attachments.Add(attachment)

    mail.Send()